goal: CCLE dataTransform_and_testing

In [1]:
import numpy as np
import pandas as pd
import os, pickle
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, GroupKFold
from sklearn.metrics import classification_report, confusion_matrix

# import tensorflow as tf # don't do this!!!
# from tensorflow import keras
from tensorflow.keras import layers, regularizers, utils, losses, optimizers
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow import linalg, random

pd.set_option("display.precision", 3)

2024-10-09 13:24:30.576099: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# load model

model = load_model("../data_temp/out_model/model_20230828.mod")

2024-10-09 13:24:37.724480: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# gdsc dict used for model training
# feature orders in TCGA should be the same as gdsc

with open('../data_temp/gdsc_dict.dict', 'rb') as f:
    gdsc_dict = pickle.load(f)

In [5]:
# double check gdsc_dict

print(gdsc_dict.keys())

dc1 = gdsc_dict['x_drug'].index == gdsc_dict["y_gdsc"].index.get_level_values(1)
print(f"dc1.all: {dc1.all()}")

dc2 = gdsc_dict["y_gdsc"].index.get_level_values(0) == gdsc_dict['x_pheno'].index
print(f"dc2.all: {dc2.all()}")

dc3 = gdsc_dict["y_gdsc"].index.get_level_values(0) == gdsc_dict['x_snv'].index
print(f"dc2.all: {dc3.all()}")

dc4 = gdsc_dict["y_gdsc"].index.get_level_values(0) == gdsc_dict['x_gep'].index
print(f"dc2.all: {dc4.all()}")

dict_keys(['y_gdsc', 'x_pheno', 'x_snv', 'x_gep', 'x_drug'])
dc1.all: True
dc2.all: True
dc2.all: True
dc2.all: True


# ccle data load

In [7]:
# 
path_ccle = "../data_input/ccle/data_20Q4"

response_mfi = "primary-screen-replicate-collapsed-logfold-change.csv"
response_mfi = pd.read_csv(os.path.join(path_ccle, response_mfi), index_col = 0)

drug_info = "primary-screen-replicate-treatment-info.csv"
drug_info = pd.read_csv(os.path.join(path_ccle, drug_info))

# note, i used pheno of the latest version, bencase of cancer tyep annotation
pheno_22Q4 = pd.read_csv("../data_input/ccle/data_22Q4/Model.csv", index_col = 0)

snv = pd.read_csv(os.path.join(path_ccle, "CCLE_mutations.csv"))
tpm = pd.read_csv(os.path.join(path_ccle, "CCLE_expression_full.csv"), index_col = 0)
# # WARNING: TMP is log2(tpm + 1), should be transformed to log2(tpm+0.001)

for v in ["pheno_22Q4", "snv", "tpm", "response_mfi", "resonse_sens", "drug_info"]:
    print(f"{v}.shape: {eval(v).shape}")

/tmp/ipykernel_5063/3574324941.py:13: DtypeWarning: Columns (3,19,22,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  snv = pd.read_csv(os.path.join(path_ccle, "CCLE_mutations.csv"))


pheno_22Q4.shape: (1826, 23)
snv.shape: (1231281, 32)
tpm.shape: (1406, 53970)
response_mfi.shape: (578, 4686)
resonse_sens.shape: (568, 4603)
drug_info.shape: (17058, 18)


In [8]:
print(snv.DepMap_ID.unique().shape)
print(tpm.shape) # row for id, column for gene

(1759,)
(1406, 53970)


In [15]:
# anno from main1_v2
df_cancerType_mapping_CCLEandTCGA = pd.read_csv("../output/df_cancerType_mapping_CCLEandTCGA.csv")
print(f"df_cancerType_mapping_CCLEandTCGA.shape: {df_cancerType_mapping_CCLEandTCGA.shape}")

# pheno_gdsc_v1 from main1_v2
pheno_gdsc_v1 = pd.read_csv("../data_temp/pheno_gdsc_v1.csv", index_col = 0)
print(f"pheno_gdsc_v1.shape: {pheno_gdsc_v1.shape}")

df_cancerType_mapping_CCLEandTCGA.shape: (180, 2)
pheno_gdsc_v1.shape: (542, 3)


# CCLE data clean: Drugs, IDs, Cancer types

## select the drugs used in the gdsc

In [16]:
# select the drugs used in the gdsc

drug_used = gdsc_dict['x_drug'].index.unique()
print(f"drug_used.shape: {drug_used.shape}")
print(f"drug_used is: {drug_used}")

regx = "|".join(drug_used)

# WARNING: the uppercase vs. lowercase
mask = drug_info.name.str.contains(regx, case = False)
drug_used_info = drug_info.loc[mask & (~mask.isna()), :]
drug_used_info = drug_used_info.loc[:, ['broad_id', 'name']].drop_duplicates()
# Note: docetaxel, Oxaliplatin has 2 broad_id, they come from different vendor
# keep only one of them

drug_used.shape: (18,)
drug_used is: Index(['Bicalutamide', 'Cisplatin', 'Cyclophosphamide', 'Dasatinib',
       'Docetaxel', 'Doxorubicin', 'Erlotinib', 'Etoposide', 'Fluorouracil',
       'Gemcitabine', 'Oxaliplatin', 'Paclitaxel', 'Pazopanib', 'Sorafenib',
       'Tamoxifen', 'Temozolomide', 'Temsirolimus', 'Vinorelbine'],
      dtype='object')


In [17]:
mask_dup = drug_used_info['name'].duplicated(keep=False)
print(drug_used_info.loc[mask_dup, :])

docetaxel_selected = "BRD-K30577245-341-01-9" # because of IC50
docetaxel_doNotUse = "BRD-K30577245-001-04-3"
oxaliplatin_selected = "BRD-K78960041-001-05-7" # randomly
oxaliplatin_doNotUse = "BRD-K78960041-001-03-2" # randomly
broadID_doNotUse = [docetaxel_doNotUse, oxaliplatin_doNotUse]
print(broadID_doNotUse)

                     broad_id         name
5914   BRD-K30577245-341-01-9    docetaxel
12675  BRD-K78960041-001-03-2  oxaliplatin
12781  BRD-K78960041-001-05-7  oxaliplatin
14128  BRD-K30577245-001-04-3    docetaxel
['BRD-K30577245-001-04-3', 'BRD-K78960041-001-03-2']


In [18]:
#
mask_dup = drug_used_info.name.duplicated(keep = False)
mask_broadID = drug_used_info.broad_id.isin(broadID_doNotUse)
drug_used_info = drug_used_info.loc[~(mask_dup & mask_broadID), :]

# revise drugs
# revise the capital letters in all the words to uppercase
drug_revise = {'5-fluorouracil': 'Fluorouracil'}
drug_remove = ['etoposide-phosphate']
mask1 = drug_used_info.name.isin(drug_remove)
drug_used_info = drug_used_info.loc[~mask1, :]
drug_used_info["drug_name_revised"] = np.where(drug_used_info.name == '5-fluorouracil',
                                              'Fluorouracil',
                                              drug_used_info.name)
drug_used_info["drug_name_revised"] = drug_used_info["drug_name_revised"].str.title()
print(f"drug_used_info.shape: {drug_used_info.shape}")

# double check
dc1 = np.sort(drug_used_info.drug_name_revised.unique()) == np.sort(drug_used)
print(f"dc1: {dc1.all()}")

drug_used_info.shape: (18, 3)
dc1: True


In [20]:
drug_used_info.sort_values(by = "drug_name_revised").head()

,broad_id,name,drug_name_revised
4634,BRD-A29485665-001-12-8,bicalutamide,Bicalutamide
13005,BRD-K69172251-001-08-9,cisplatin,Cisplatin
8202,BRD-A09722536-002-18-0,cyclophosphamide,Cyclophosphamide
1426,BRD-K49328571-001-15-0,dasatinib,Dasatinib
5914,BRD-K30577245-341-01-9,docetaxel,Docetaxel


In [15]:
# save
# drug_used_info.to_csv("../output/out_ccle/v20230812/drug_used_info.csv", index = False)

## select IDs that are unique to ccle

In [21]:
## select cells that are unique to ccle

id_gdsc = gdsc_dict["y_gdsc"].index.get_level_values(0)
print(f"id_gdsc.unique.shape: {id_gdsc.unique().shape}")
print(id_gdsc.unique())

mask1 = pheno_22Q4.SangerModelID.isin(id_gdsc)
print(f"mask1.sum: {mask1.sum()}")
pheno_ccle = pheno_22Q4.loc[~mask1, :]
print(f"pheno_ccle.shape: {pheno_ccle.shape}")
print(f"pheno_22Q4.shape: {pheno_22Q4.shape}")

id_gdsc.unique.shape: (542,)
Index(['SIDM00003', 'SIDM00023', 'SIDM00042', 'SIDM00043', 'SIDM00044',
       'SIDM00046', 'SIDM00049', 'SIDM00050', 'SIDM00051', 'SIDM00052',
       ...
       'SIDM01189', 'SIDM01191', 'SIDM01197', 'SIDM01210', 'SIDM01228',
       'SIDM01233', 'SIDM01240', 'SIDM01242', 'SIDM01246', 'SIDM01248'],
      dtype='object', name='SANGER_MODEL_ID', length=542)
mask1.sum: 542
pheno_ccle.shape: (1284, 23)
pheno_22Q4.shape: (1826, 23)


In [22]:
# filter cancer type
pheno_ccle_v1 = pd.merge(pheno_ccle.reset_index(), df_cancerType_mapping_CCLEandTCGA,
                        left_on = "OncotreeCode", right_on = "OncotreeCode_CCLE",
                        how = "inner")
print(f"pheno_ccle_v1.shape: {pheno_ccle_v1.shape}")

mask = pheno_ccle_v1.cancerType_TCGA.isin(pheno_gdsc_v1.cancerType_TCGA.unique())
pheno_ccle_v1 = pheno_ccle_v1.loc[mask, :]
print(f"pheno_ccle_v1.shape: {pheno_ccle_v1.shape}")

pheno_ccle_v1.shape: (1177, 26)
pheno_ccle_v1.shape: (382, 26)


In [23]:
#  id_inter
from functools import reduce

id_response = response_mfi.index.unique()
id_pheno = pheno_ccle_v1.ModelID.unique()
id_snv = snv.DepMap_ID.unique()
id_gep = tpm.index.unique()

id_inter4 = reduce(np.intersect1d, [id_response, id_pheno, id_snv, id_gep])
print(f"id_inter4.shape: {id_inter4.shape}")

id_inter4.shape: (93,)


In [20]:
# save
# np.save("../output/out_ccle/v20230812/id_inter4", id_inter4)

# CCLE data transform: response, pheno, snv, gep, drug

## response

In [24]:
response_mfi.head(2)

,BRD-A00077618-236-07-6::2.5::HTS,BRD-A00100033-001-08-9::2.5::HTS,BRD-A00147595-001-01-5::2.5::HTS,BRD-A00218260-001-03-4::2.5::HTS,BRD-A00376169-001-01-6::2.5::HTS,BRD-A00520476-001-07-4::2.5::HTS,BRD-A00546892-001-02-6::2.5::HTS,BRD-A00578795-001-04-3::2.5::HTS,BRD-A00758722-001-04-9::2.5::HTS,BRD-A00827783-001-24-6::2.5::HTS,...,BRD-K98557884-001-01-6::2.5::MTS004,BRD-K99077012-001-01-9::2.332734192::MTS004,BRD-K99199077-001-16-1::2.603211317::MTS004,BRD-K99431849-001-01-7::2.500018158::MTS004,BRD-K99447003-335-04-1::2.37737659::MTS004,BRD-K99506538-001-03-8::2.5::MTS004,BRD-K99616396-001-05-1::2.499991421::MTS004,BRD-K99879819-001-02-1::2.5187366::MTS004,BRD-K99919177-001-01-3::2.5::MTS004,BRD-M63173034-001-03-6::2.64076472::MTS004
ACH-000001,-0.016,-0.449,0.489,0.207,0.273,0.021,-0.025,0.467,-0.736,0.644,...,0.429,0.205,0.150,-0.575,-0.101,0.399,-0.128,-0.142,-1.154,0.510
ACH-000007,-0.096,0.258,0.772,-0.439,-0.733,0.779,0.427,-1.289,-0.476,-0.277,...,-0.471,0.213,-0.123,0.626,0.383,0.212,0.349,-0.387,-0.831,0.324


In [25]:
# transfer response

# revise columns
t1 = response_mfi.columns.str.extract("([^:]+)")[0]
print(f"Check dup: {t1.duplicated().any()}")
response_mfi.columns = t1
mask_drug = response_mfi.columns.isin(drug_used_info.broad_id)
mask_cell = response_mfi.index.isin(id_inter4)
response_mfi_v1 = response_mfi.loc[mask_cell, mask_drug]
print(f"response_mfi_v1.shape: {response_mfi_v1.shape}")


# fillna by mean value
num_NAs = response_mfi_v1.isna().sum().sum()
print(f"num_NAs: {num_NAs}")

response_mfi_v1 = response_mfi_v1.fillna(response_mfi_v1.mean())
num_NAs = response_mfi_v1.isna().sum().sum()
print(f"num_NAs: {num_NAs}")

# transform
y = response_mfi_v1.stack()
# you must set names first
y.name = "log2FC"
y.index.names = ["DepMap_ID", "broad_id"]
y = y.reset_index()
y = pd.merge(y, drug_used_info, left_on = "broad_id", right_on = "broad_id", how = "inner")
print(f"y.shape: {y.shape}")

# double check
dc1 = y.drug_name_revised.unique().shape
print(f"dc1.shape: {dc1}")

# 
y_ccle = y.set_index(["DepMap_ID", "drug_name_revised"]).log2FC
print(y_ccle.shape)

Check dup: False
response_mfi_v1.shape: (93, 18)
num_NAs: 52
num_NAs: 0
y.shape: (1674, 5)
dc1.shape: (18,)
(1674,)


In [26]:
y_ccle.head()

DepMap_ID   drug_name_revised
ACH-000013  Cyclophosphamide     0.252
ACH-000014  Cyclophosphamide     0.523
ACH-000022  Cyclophosphamide    -0.504
ACH-000027  Cyclophosphamide     0.341
ACH-000037  Cyclophosphamide     0.573
Name: log2FC, dtype: float64

In [24]:
y.head(2)

,DepMap_ID,broad_id,log2FC,name,drug_name_revised
0,ACH-000013,BRD-A09722536-002-18-0,0.252,cyclophosphamide,Cyclophosphamide
1,ACH-000014,BRD-A09722536-002-18-0,0.523,cyclophosphamide,Cyclophosphamide


In [25]:
# # save
# y_ccle.to_csv("../output/out_ccle/v20230812/y_ccle.csv", index = True)
# y.to_csv("../output/out_ccle/v20230812/y_ccle_anno.csv", index = False)

## Pheno

In [27]:
pheno_ccle_v1.head(2)

,ModelID,PatientID,CellLineName,StrippedCellLineName,Age,SourceType,SangerModelID,RRID,DepmapModelType,GrowthPattern,...,CCLEName,COSMICID,PublicComments,WTSIMasterCellID,OncotreeCode,OncotreeSubtype,OncotreePrimaryDisease,OncotreeLineage,OncotreeCode_CCLE,cancerType_TCGA
36,ACH-000003,PT-puKIyc,CACO2,CACO2,72.0,Commercial,SIDM00891,CVCL_0025,COAD,Adherent,...,CACO2_LARGE_INTESTINE,NaN,NaN,NaN,COAD,Colon Adenocarcinoma,Colorectal Adenocarcinoma,Bowel,COAD,COAD
37,ACH-000089,PT-0joF2E,NCI-H684,NCIH684,55.0,Commercial,NaN,CVCL_9980,COAD,Unknown,...,NCIH684_LARGE_INTESTINE,NaN,NaN,NaN,COAD,Colon Adenocarcinoma,Colorectal Adenocarcinoma,Bowel,COAD,COAD


In [28]:
# pheno process

# ID process
mask_id = pheno_ccle_v1.ModelID.isin(id_inter4)
col_use = ['ModelID', 'Age', 'Sex', 'cancerType_TCGA']
pheno_ccle_v2 = pheno_ccle_v1.loc[mask_id, col_use].drop_duplicates()
print("pheno_ccle_v2.shape: {}".format(pheno_ccle_v2.shape))

# process NAs
print(pheno_ccle_v2.Sex.value_counts())
mask_age = pheno_ccle_v2.Age.isna()
mask_sex = (pheno_ccle_v2.Sex == "Unknown")
print(f"# NAs in Age: {mask_age.sum()}")
print(f"# NAs in sex: {mask_sex.sum()}")

# fill NAs in sex by random, in age by mean age

rng = np.random.default_rng(0)
x = rng.choice(["Male", "Female"], size=5, replace=True, 
           p=[54/(54+34), 34/(54+34)], axis=0, shuffle=True)
pheno_ccle_v2 = pheno_ccle_v2.sort_values("Sex", ascending = False)
pheno_ccle_v2["Sex"].iloc[0:5] = x

age_median = pheno_ccle_v2.Age.median().astype(int)
pheno_ccle_v2["Age"] = pheno_ccle_v2["Age"].fillna(pheno_ccle_v2["Age"].median())
print(pheno_ccle_v2.Age.describe())

# double check NAs
check1 = pheno_ccle_v2.apply(lambda x: x.isna().sum() == 0)
print(check1)

pheno_ccle_v2.shape: (93, 4)
Male       54
Female     34
Unknown     5
Name: Sex, dtype: int64
# NAs in Age: 22
# NAs in sex: 5
count    93.000
mean     54.946
std      13.719
min       1.000
25%      53.000
50%      57.000
75%      62.000
max      88.000
Name: Age, dtype: float64
ModelID            True
Age                True
Sex                True
cancerType_TCGA    True
dtype: bool


/tmp/ipykernel_5063/4150016031.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pheno_ccle_v2["Sex"].iloc[0:5] = x


In [29]:
# dummy and and diff_cancer types
pheno_ccle_v2 = pheno_ccle_v2.set_index("ModelID").loc[id_inter4, :]
pheno_ccle_v2_dummy = pd.get_dummies(data = pheno_ccle_v2, columns = ["Sex", "cancerType_TCGA"])
print(f"pheno_ccle_v2.shape: {pheno_ccle_v2.shape}")
print(f"pheno_ccle_v2_dummy.shape: {pheno_ccle_v2_dummy.shape}")
print(f"pheno_ccle_v2_dummy.columns: {pheno_ccle_v2_dummy.columns}")


# add col_diff to pheno_tcga_dummy
col_diff = np.setdiff1d(gdsc_dict['x_pheno'].columns, pheno_ccle_v2_dummy.columns)
print(f"col_diff.shape: {col_diff.shape}")
print(f"col_diff: {col_diff}")

pheno_add = pd.DataFrame(np.zeros(shape=(pheno_ccle_v2_dummy.shape[0], len(col_diff))), 
                         columns = col_diff,
                        index = pheno_ccle_v2_dummy.index)
print(f"pheno_add.shape: {pheno_add.shape}")
pheno_ccle_v2_dummy = pd.concat([pheno_ccle_v2_dummy, pheno_add], axis = 1)
print(f"pheno_ccle_v2_dummy.shape: {pheno_ccle_v2_dummy.shape}")

pheno_ccle_v2.shape: (93, 3)
pheno_ccle_v2_dummy.shape: (93, 14)
pheno_ccle_v2_dummy.columns: Index(['Age', 'Sex_Female', 'Sex_Male', 'cancerType_TCGA_BRCA',
       'cancerType_TCGA_COAD', 'cancerType_TCGA_ESCA', 'cancerType_TCGA_GBM',
       'cancerType_TCGA_HNSC', 'cancerType_TCGA_KIRC', 'cancerType_TCGA_LUAD',
       'cancerType_TCGA_OV', 'cancerType_TCGA_PAAD', 'cancerType_TCGA_SARC',
       'cancerType_TCGA_SKCM'],
      dtype='object')
col_diff.shape: (1,)
col_diff: ['cancerType_TCGA_SCLC']
pheno_add.shape: (93, 1)
pheno_ccle_v2_dummy.shape: (93, 15)


In [29]:
pheno_ccle_v2_dummy.head(2)

,Age,Sex_Female,Sex_Male,cancerType_TCGA_BRCA,cancerType_TCGA_COAD,cancerType_TCGA_ESCA,cancerType_TCGA_GBM,cancerType_TCGA_HNSC,cancerType_TCGA_KIRC,cancerType_TCGA_LUAD,cancerType_TCGA_OV,cancerType_TCGA_PAAD,cancerType_TCGA_SARC,cancerType_TCGA_SKCM,cancerType_TCGA_SCLC
ModelID,,,,,,,,,,,,,,,
ACH-000013,60.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0.0
ACH-000014,56.0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.0


In [30]:
# save
pheno_ccle_v2.to_csv("../data_temp/out_ccle/v20230812/pheno_ccle_v2.csv", index = True)
pheno_ccle_v2_dummy.to_csv("../data_temp/out_ccle/v20230812/pheno_ccle_v2_dummy.csv", index = True)

## SNV

In [30]:
snv.head(2)

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,isCOSMIChotspot,COSMIChsCnt,ExAC_AF,Variant_annotation,CGA_WES_AC,HC_AC,RD_AC,RNAseq_AC,SangerWES_AC,WGS_AC
0,VPS13D,55187,37,1,12359347,12359347,+,Nonsense_Mutation,SNP,C,...,False,0.0,NaN,damaging,34:213,NaN,NaN,NaN,34:221,NaN
1,AADACL4,343066,37,1,12726308,12726322,+,In_Frame_Del,DEL,CTGGCGTGACGCCAT,...,False,3.0,NaN,other non-conserving,57:141,NaN,NaN,NaN,9:0,28:32


In [31]:
# All genes used in gdsc
genes_full = gdsc_dict['x_snv'].columns.unique()
print(f"genes_full: {genes_full.shape}")

genes_full: (776,)


In [32]:
# transfer snv

mask_gene = snv.Hugo_Symbol.isin(genes_full)
mask_cell = snv.DepMap_ID.isin(id_inter4)

vc_list = [
    'Nonsense_Mutation', 'In_Frame_Del', 'Frame_Shift_Ins',
    'Missense_Mutation', 'Splice_Site', 'Frame_Shift_Del',
    'De_novo_Start_OutOfFrame', 'Nonstop_Mutation', 'In_Frame_Ins',
    'Start_Codon_SNP', 'Start_Codon_Del', 'Stop_Codon_Ins', 'Start_Codon_Ins',
    'Stop_Codon_Del'
]
mask_vc = snv.Variant_Classification.isin(vc_list)

snv_v1 = snv.loc[mask_gene & mask_cell & mask_vc, :]
print(f"snv_v1.shape: {snv_v1.shape}")

snv_v1['i_mut'] = 1.0
snv_v1 = snv_v1.drop_duplicates(["DepMap_ID", "Hugo_Symbol"])
snv_v1 = snv_v1.pivot(index="DepMap_ID", columns="Hugo_Symbol", values="i_mut")
snv_v1 = snv_v1.fillna(0)
print(f"snv_v1.shape: {snv_v1.shape}")

id_lack = np.setdiff1d(id_inter4, snv_v1.index)
if len(id_lack) > 0:
    add_id = pd.DataFrame(np.zeros((id_lack.shape[0], snv_v1.shape[0])),
                          index=id_lack,
                          columns=snv_v1.columns)
    print(f"add_id.shape: {add_id.shape}")
    snv_v1 = pd.concat([snv_v1, add_id], axis=0)
    print(f"After add id: snv_v1.shape: {snv_v1.shape}")

gene_miss = np.setdiff1d(genes_full, snv_v1.columns)
if len(gene_miss) > 0:
    add_gene = pd.DataFrame(np.zeros((snv_v1.shape[0], gene_miss.shape[0])),
                            columns=gene_miss,
                            index=snv_v1.index)
    print(f"add_gene.shape: {add_gene.shape}")
    snv_v1 = pd.concat([snv_v1, add_gene], axis=1)
    print(f"After add gene: snv_v1.shape: {snv_v1.shape}")

snv_v1.shape: (2894, 32)
snv_v1.shape: (93, 644)
add_gene.shape: (93, 132)
After add gene: snv_v1.shape: (93, 776)


/tmp/ipykernel_5063/104991296.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snv_v1['i_mut'] = 1.0


In [35]:
# save
snv_v1.to_csv("../data_temp/out_ccle/v20230812/snv_v1.csv", index = True)

## TPM

In [33]:
tpm.head(2)

,TSPAN6 (ENSG00000000003),TNMD (ENSG00000000005),DPM1 (ENSG00000000419),SCYL3 (ENSG00000000457),C1orf112 (ENSG00000000460),FGR (ENSG00000000938),CFH (ENSG00000000971),FUCA2 (ENSG00000001036),GCLC (ENSG00000001084),NFYA (ENSG00000001167),...,ENSG00000288714,ENSG00000288717,ENSG00000288718,ENSG00000288719,ENSG00000288720,ENSG00000288721,ENSG00000288722,ENSG00000288723,ENSG00000288724,ENSG00000288725
ACH-001113,4.332,0.000,7.364,2.793,4.471,0.029,1.227,3.043,6.500,4.740,...,0.0,0.536,0.0,0.029,0.176,0.993,2.795,0.000,0.0,0.00
ACH-001289,4.567,0.585,7.107,2.543,3.505,0.000,0.189,3.814,4.221,3.482,...,0.0,0.880,0.0,0.014,0.014,0.433,2.973,0.057,0.0,0.07


In [35]:
# transform tpm from log2(tpm+1) to log2(tpm+0.001)

def trans_log(x):
    tpm = 2**x - 1
    y = np.log2(tpm + 0.001)
    return y
tpm_v1 = tpm.apply(trans_log)

In [36]:
# double check
tpm_v1.min().value_counts()

-9.966    45281
-6.506      446
-5.573      314
-5.012      233
-4.608      180
          ...  
 4.261        1
 3.811        1
 5.280        1
 3.178        1
 6.190        1
Length: 1389, dtype: int64

In [37]:
# process column names and duplications
tpm_v1 = tpm_v1.stack().reset_index()
tpm_v1['gene_symbol'] = tpm_v1.level_1.str.extract("([^ (]+)", expand = False).astype(str)
tpm_v1 = tpm_v1.groupby(['level_0', 'gene_symbol'])[[0]].mean()
tpm_v1 = tpm_v1.unstack('gene_symbol')
tpm_v1.columns = tpm_v1.columns.droplevel(level=0)
print(f"tpm_v1 clean.shape: {tpm_v1.shape}")

# filter by id_inter4 and genes_full
tpm_v1 = tpm_v1.loc[tpm_v1.index.isin(id_inter4), tpm_v1.columns.isin(genes_full)]
print(f"tpm_v1 after filtering.shape: {tpm_v1.shape}")

tpm_v1 clean.shape: (1406, 53960)
tpm_v1 after filtering.shape: (93, 776)


In [38]:
# double check
tpm_v1.min().value_counts()

-9.966    184
-6.506     28
-5.573     14
-4.293     14
-5.012     11
         ... 
 1.345      1
 1.428      1
 2.043      1
 4.319      1
-0.072      1
Length: 430, dtype: int64

In [39]:
tpm_v1.head(2)

gene_symbol,ABCB1,ABI1,ABL1,ABL2,ACKR3,ACSL3,ACVR1,ACVR2A,AFDN,AFF1,...,ZNF521,ZNF626,ZNF680,ZNF721,ZNF780A,ZNF814,ZNF93,ZNRF3,ZRSR2,ZXDB
level_0,,,,,,,,,,,,,,,,,,,,,
ACH-000013,-1.781,3.424,4.611,2.325,-0.597,6.148,3.032,2.685,5.926,2.592,...,-6.506,-1.392,2.678,4.124,4.852,0.934,-0.472,3.558,2.057,1.585
ACH-000014,0.576,4.057,4.240,4.943,2.808,6.028,4.642,1.714,3.940,2.858,...,2.211,-2.548,0.215,3.985,3.297,2.278,-1.831,2.015,3.964,0.633


In [42]:
# save
tpm_v1.to_csv("../data_temp/out_ccle/v20230812/tpm_v1.csv", index = True)

## transform pheno, snv, tpm, and drug

In [40]:
df_drug = pd.read_csv("../data_temp/df_fingerprints_18drugs.csv", index_col = 0)
print(f"df_drug.shape: {df_drug.shape}")

df_drug.shape: (18, 512)


In [41]:
tpm_v1.head(2)

gene_symbol,ABCB1,ABI1,ABL1,ABL2,ACKR3,ACSL3,ACVR1,ACVR2A,AFDN,AFF1,...,ZNF521,ZNF626,ZNF680,ZNF721,ZNF780A,ZNF814,ZNF93,ZNRF3,ZRSR2,ZXDB
level_0,,,,,,,,,,,,,,,,,,,,,
ACH-000013,-1.781,3.424,4.611,2.325,-0.597,6.148,3.032,2.685,5.926,2.592,...,-6.506,-1.392,2.678,4.124,4.852,0.934,-0.472,3.558,2.057,1.585
ACH-000014,0.576,4.057,4.240,4.943,2.808,6.028,4.642,1.714,3.940,2.858,...,2.211,-2.548,0.215,3.985,3.297,2.278,-1.831,2.015,3.964,0.633


In [42]:
# TRANSFORM: snv, tpm, pheno, dug
# extract the order of sample and drug
order_sample = y_ccle.index.get_level_values(0)
order_drug = y_ccle.index.get_level_values(1)
order_gene = gdsc_dict['x_snv'].columns
order_pheno = gdsc_dict['x_pheno'].columns
order_drug_features = gdsc_dict['x_drug'].columns

# x_snv
x_snv = snv_v1.loc[order_sample, order_gene]
print(f"x_snv.shape: {x_snv.shape}")

# gep
x_gep = tpm_v1.loc[order_sample, order_gene]
print(f"x_gep.shape: {x_gep.shape}")

# pheno
x_pheno = pheno_ccle_v2_dummy.loc[order_sample, order_pheno]
print(f"x_pheno.shape: {x_pheno.shape}")

# drug
x_drug = df_drug.loc[order_drug, order_drug_features]
print(f"x_drug.shape: {x_drug.shape}")

x_snv.shape: (1674, 776)
x_gep.shape: (1674, 776)
x_pheno.shape: (1674, 15)
x_drug.shape: (1674, 512)


In [46]:
# save dict

ccle_dict = {
    "y": y_ccle,
    "x_pheno": x_pheno,
    "x_snv": x_snv,
    "x_gep": x_gep,
    "x_drug": x_drug
}

# save
with open('../data_temp/out_ccle/v20230812/ccle_dict.dict', 'wb') as f:
    pickle.dump(ccle_dict, f)
    
# with open("../data_temp/out_ccle/v20230812/ccle_dict.dict", "rb") as f:
#     ccle_dict = pickle.load(f)

# model test

In [47]:
# with open("../output/out_ccle/v20230812/ccle_dict.dict", "rb") as f:
#     ccle_dict = pickle.load(f)
# pheno_ccle_v2 = pd.read_csv("../data_temp/out_ccle/v20230812/pheno_ccle_v2.csv", index_col = 0)

In [43]:
pheno_ccle_v2.head(2)

,Age,Sex,cancerType_TCGA
ModelID,,,
ACH-000013,60.0,Female,OV
ACH-000014,56.0,Male,SKCM


In [50]:
xs = ['x_pheno', 'x_snv', 'x_gep', 'x_drug']
xs = [ccle_dict[e].astype('float64').to_numpy() for e in xs]
y_pred_ccle = model.predict(xs, batch_size = 64)

27/27 [==============================] - 0s 1ms/step


In [51]:
# y_ccle_anno
y_pred_v1 = pd.concat([ccle_dict['y'].reset_index(),
                       pd.Series(y_pred_ccle.flatten(), name = "ypred")], 
                      axis = 1)
print(f"y_pred_v1.shape: {y_pred_v1.shape}")

y_pred_v1_anno = pd.merge(y_pred_v1,
                      pheno_ccle_v2,
                       left_on = "DepMap_ID",
                       right_index = True,
                       how = "inner")
print(f"y_pred_v1_anno.shape: {y_pred_v1_anno.shape}")

# double check
dc = np.sort(y_pred_v1.DepMap_ID.values) == y_pred_v1_anno.DepMap_ID.values
print(f"dc.all: {dc.all()}")

y_pred_v1.shape: (1674, 4)
y_pred_v1_anno.shape: (1674, 7)
dc.all: True


In [52]:
y_pred_v1_anno.head()

,DepMap_ID,drug_name_revised,log2FC,ypred,Age,Sex,cancerType_TCGA
0,ACH-000013,Cyclophosphamide,0.252,0.994,60.0,Female,OV
93,ACH-000013,Bicalutamide,0.055,0.439,60.0,Female,OV
186,ACH-000013,Gemcitabine,-1.424,-0.572,60.0,Female,OV
279,ACH-000013,Sorafenib,-0.145,0.152,60.0,Female,OV
372,ACH-000013,Fluorouracil,-0.520,0.734,60.0,Female,OV


In [53]:
y_pred_v1_pivot = y_pred_v1_anno.pivot_table(
    values = "ypred", 
    index = "DepMap_ID", 
    columns = "drug_name_revised")

drug_sens = y_pred_v1_pivot.apply(lambda x: y_pred_v1_pivot.columns[np.argmin(x)], axis = 1).value_counts()
print(f"drug_sens: \n{drug_sens}")

drug_sens: 
Docetaxel    93
dtype: int64


In [54]:
# save
y_pred_v1_anno.to_csv("../data_temp/out_ccle/v20230812/y_pred_v1_anno_20230828.csv", index = False)
y_pred_v1_pivot.to_csv("../data_temp/out_ccle/v20230812/y_pred_v1_pivot_anno_20230828.csv", index = True)

In [55]:
y_pred_v1_anno.head(2)

,DepMap_ID,drug_name_revised,log2FC,ypred,Age,Sex,cancerType_TCGA
0,ACH-000013,Cyclophosphamide,0.252,0.994,60.0,Female,OV
93,ACH-000013,Bicalutamide,0.055,0.439,60.0,Female,OV


In [56]:
# correlation between ytrue and ypred

corr_overall = y_pred_v1_anno['log2FC'].corr(y_pred_v1_anno['ypred'])
print(f"corr_overall: {corr_overall}")

for k, v in y_pred_v1_anno.groupby("drug_name_revised"):
    cor_coef = v['log2FC'].corr(v['ypred'])
    print(f"correlation score in drug {k} is {cor_coef}") 

corr_overall: 0.6550494545293325
correlation score in drug Bicalutamide is 0.12080672107190019
correlation score in drug Cisplatin is 0.11275370023547297
correlation score in drug Cyclophosphamide is 0.11839158503237097
correlation score in drug Dasatinib is 0.07464897038626397
correlation score in drug Docetaxel is 0.16414448106038132
correlation score in drug Doxorubicin is -0.054575048628070735
correlation score in drug Erlotinib is 0.25656679871650284
correlation score in drug Etoposide is 0.2612851860529751
correlation score in drug Fluorouracil is 0.18499221159757587
correlation score in drug Gemcitabine is 0.24218124509536462
correlation score in drug Oxaliplatin is 0.37483670584040024
correlation score in drug Paclitaxel is 0.3353750789367856
correlation score in drug Pazopanib is 0.0376874343295385
correlation score in drug Sorafenib is 0.16925134125237382
correlation score in drug Tamoxifen is 0.005966469809325718
correlation score in drug Temozolomide is -0.08832568590974495